# Main

## 1) Loading the data

In [45]:
import pandas as pd
import numpy as np

data = pd.read_csv('data\kidney_disease.csv')

data 

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,49,6600,5.4,no,no,no,good,no,no,notckd
398,398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,51,7200,5.9,no,no,no,good,no,no,notckd


In [46]:
# Decode strings
data.columns = data.columns.str.strip()

# Replace '?' with NaN
data.replace('?', np.nan)

# Convert columns to numeric, forcing errors to NaN
data = data.apply(pd.to_numeric, errors='ignore')

# Print the data types to verify the conversion
# print(data.dtypes)

# Convert nominal data to 'category' data type
categorical_features = ['sg','al','su','rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane'] # the 14 nominal attributes minus 'class'
for feature in categorical_features:
    data[feature] = data[feature].astype('category')

# Verify the conversion
# print(data.dtypes)

# Remove the "ground-truth" column and store its values aside
ground_truth = data['classification']
data = data.drop(columns=['classification'])

# Verify the removal
# print(data.head())
# print(ground_truth.head())

# Replace missing numeric values with the feature average
numeric_features = data.select_dtypes(exclude='category').columns
for feature in numeric_features:
    data[feature] = pd.to_numeric(data[feature], errors='coerce')  # Convert to numeric to avoid errors
    data[feature].fillna(data[feature].mean(), inplace=True)

# Replace missing categorical values with the most occurring value
for feature in categorical_features:
    most_frequent_value = data[feature].mode().iloc[0]
    data[feature].fillna(most_frequent_value, inplace=True)

# # Verify that there are no more NaN values
# nan_counts = data.isnull().sum()
# # print("Number of NaN values in each column:")
# # print(nan_counts)

# # Check if there are any columns with NaN values
# if nan_counts.sum() == 0:
#     print("There are no more NaN values in the DataFrame.")
# else:
#     print("There are still NaN values in the DataFrame.")

# One-hot encode categorical features
data_encoded = pd.get_dummies(data, columns=categorical_features)

# Verify the encoding
# print(data_encoded.head())

# Convert boolean columns to integers (0 and 1)
boolean_columns = data_encoded.select_dtypes(include=['bool']).columns
data_encoded[boolean_columns] = data_encoded[boolean_columns].astype('int')

# print(data_encoded.head())

data_normalized = (data_encoded - data_encoded.mean()) / data_encoded.std()

# Convert the "ground-truth" data into numeric format
ground_truth_numeric = ground_truth.map(lambda x: 1 if x == 'ckd' else 0).to_numpy()

Y=ground_truth_numeric

X = data_normalized.to_numpy()

# Combine X and Y into a single dataset
data = np.column_stack((X, Y))


C:\Users\donoa\AppData\Local\Temp\ipykernel_50208\558504442.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = data.apply(pd.to_numeric, errors='ignore')
C:\Users\donoa\AppData\Local\Temp\ipykernel_50208\558504442.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[feature].fillna(data[feature].mean(), inplace=True)
C:\Users\donoa\AppData\Local\Temp\ipykernel_50208\558504442.py:38: FutureWarning: A

In [47]:
data

array([[-1.72556507, -0.20520665,  0.26201023, ...,  0.41955859,
        -0.41955859,  1.        ],
       [-1.71691562, -2.6205281 , -1.96412048, ...,  0.41955859,
        -0.41955859,  1.        ],
       [-1.70826617,  0.61953726,  0.26201023, ..., -2.37749869,
         2.37749869,  1.        ],
       ...,
       [ 1.70826617, -2.3259767 ,  0.26201023, ...,  0.41955859,
        -0.41955859,  0.        ],
       [ 1.71691562, -2.03142531, -1.22207691, ...,  0.41955859,
        -0.41955859,  0.        ],
       [ 1.72556507,  0.38389614,  0.26201023, ...,  0.41955859,
        -0.41955859,  0.        ]])

In [48]:
data_banknote = pd.read_csv('data\data_banknote_authentication.txt', names=['variance', 'skewness', 'curtosis', 'entropy', 'class'])

In [49]:
data_banknote

,variance,skewness,curtosis,entropy,class
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


In [50]:
ground_truth_banknote = data_banknote['class']
data_banknote = data_banknote.drop(columns=['class'])

In [51]:
nan_counts = data_banknote.isnull().sum()
print("Number of NaN values in each column:")
print(nan_counts)

# Check if there are any columns with NaN values
if nan_counts.sum() == 0:
    print("There are no more NaN values in the DataFrame.")
else:
    print("There are still NaN values in the DataFrame.")

Number of NaN values in each column:
variance    0
skewness    0
curtosis    0
entropy     0
dtype: int64
There are no more NaN values in the DataFrame.


In [52]:
data_banknote_normalized = (data_banknote - data_banknote.mean()) / data_banknote.std()

In [53]:
data_banknote_normalized

,variance,skewness,curtosis,entropy
0,1.121397,1.149036,-0.975614,0.354432
1,1.446538,1.064065,-0.894710,-0.128721
2,1.207369,-0.777069,0.122174,0.617848
3,1.063355,1.295005,-1.254940,-1.143612
4,-0.036758,-1.086642,0.736462,0.096552
...,...,...,...,...
1367,-0.009707,-0.097657,-0.660721,0.300887
1368,-0.641079,-1.158562,1.178593,0.729861
1369,-1.471820,-2.620690,3.757647,-0.754609
1370,-1.406180,-1.755831,2.551113,-0.043143


In [54]:
Y=ground_truth_banknote.to_numpy()

X = data_banknote_normalized.to_numpy()

# Combine X and Y into a single dataset
data_banknote = np.column_stack((X, Y))

In [55]:
data_banknote

array([[ 1.12139676,  1.14903615, -0.97561433,  0.35443211,  0.        ],
       [ 1.44653822,  1.06406494, -0.89471002, -0.12872051,  0.        ],
       [ 1.20736947, -0.7770688 ,  0.12217383,  0.61784788,  0.        ],
       ...,
       [-1.47182015, -2.62069017,  3.75764729, -0.75460903,  1.        ],
       [-1.40617978, -1.75583081,  2.55111279, -0.04314275,  1.        ],
       [-1.04674069, -0.43966137,  0.29850671,  1.13605026,  1.        ]])

In [56]:
data 

array([[-1.72556507, -0.20520665,  0.26201023, ...,  0.41955859,
        -0.41955859,  1.        ],
       [-1.71691562, -2.6205281 , -1.96412048, ...,  0.41955859,
        -0.41955859,  1.        ],
       [-1.70826617,  0.61953726,  0.26201023, ..., -2.37749869,
         2.37749869,  1.        ],
       ...,
       [ 1.70826617, -2.3259767 ,  0.26201023, ...,  0.41955859,
        -0.41955859,  0.        ],
       [ 1.71691562, -2.03142531, -1.22207691, ...,  0.41955859,
        -0.41955859,  0.        ],
       [ 1.72556507,  0.38389614,  0.26201023, ...,  0.41955859,
        -0.41955859,  0.        ]])